## Load Data

In [1]:
import pickle

with open('D:/Shetty_data/train/uav_image_data.pickle', 'rb') as handle:
    matching_uav_data = pickle.load(handle)

with open('D:/Shetty_data/train/uav_image_data.pickle', 'rb') as handle:
    non_matching_uav_data = pickle.load(handle)

with open('D:/Shetty_data/train/matching_sat_data.pickle', 'rb') as handle:
    matching_sat_data = pickle.load(handle)

with open('D:/Shetty_data/train/non_matching_sat_data.pickle', 'rb') as handle:
    non_matching_sat_data = pickle.load(handle)


## Loss Function

In [2]:
import numpy as np

def contrastive_loss(d,l):
    m = 100
    loss = l*torch.pow(d, 2) + (1-l) * torch.pow(torch.clamp(m - d, min=0.0), 2)
    return torch.sum(loss)/d.size()[0]

## Create training tensors

In [3]:
import torch

matching_uav_data = torch.stack(matching_uav_data)
non_matching_uav_data = torch.stack(non_matching_uav_data)
matching_sat_data = torch.stack(matching_sat_data)
non_matching_sat_data = torch.stack(non_matching_sat_data)

## Divide data into training and validation

In [4]:
matching_uav_data_validation = matching_uav_data[int(0.9*len(matching_uav_data)):]
non_matching_uav_data_validation = non_matching_uav_data[int(0.9*len(non_matching_uav_data)):]
matching_sat_data_validation = matching_sat_data[int(0.9*len(matching_sat_data)):]
non_matching_sat_data_validation = non_matching_sat_data[int(0.9*len(non_matching_sat_data)):]


matching_uav_data = matching_uav_data[:int(0.9*len(matching_uav_data))]
non_matching_uav_data = non_matching_uav_data[:int(0.9*len(non_matching_uav_data))]
matching_sat_data = matching_sat_data[:int(0.9*len(matching_sat_data))]
non_matching_sat_data = non_matching_sat_data[:int(0.9*len(non_matching_sat_data))]

## Train network

In [5]:
from torchvision import datasets, models, transforms
from scene_network_alexnet import alexnet_siamese as scene_network
import torch.optim as optim
from sklearn.utils import shuffle
from scene_network_resnet import scene_network as resnet

# scene_model = resnet("resnet18")
scene_model = scene_network()
optimizer = optim.Adam(scene_model.parameters(), lr=10e-5)

scene_model.train()

uav_validation_data = torch.cat((matching_uav_data_validation,non_matching_uav_data_validation))

sat_validation_data = torch.cat((matching_sat_data_validation,non_matching_sat_data_validation))

validation_labels = torch.tensor([1]*matching_uav_data_validation.size()[0]+[0]*non_matching_uav_data_validation.size()[0])

epochs = 10
batch_size = 32

for epoch in range(epochs):

    matching_uav_data,matching_sat_data = shuffle(matching_uav_data,matching_sat_data)

    non_matching_uav_data,non_matching_sat_data = shuffle(non_matching_uav_data,non_matching_sat_data)

    running_loss = 0.0

    for i in range(len(matching_sat_data)//batch_size):

        uav_input = torch.cat((matching_uav_data[i*(batch_size//2):(i+1)*(batch_size//2)],non_matching_uav_data[i*(batch_size//2):(i+1)*(batch_size//2)]))

        sat_input = torch.cat((matching_sat_data[i*(batch_size//2):(i+1)*(batch_size//2)],non_matching_sat_data[i*(batch_size//2):(i+1)*(batch_size//2)]))


        optimizer.zero_grad()

        l = torch.tensor(([1]*(batch_size//2))+[0]*(batch_size//2))

        d = scene_model(uav_input,sat_input)

        loss = contrastive_loss(d, l)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
            
            d_val = scene_model(uav_validation_data,sat_validation_data)

            val_loss = contrastive_loss(d_val,validation_labels)
            print("Val loss", val_loss)



[1,    10] loss: 1816.048
Val loss tensor(1995.9780, grad_fn=<DivBackward0>)
[1,    20] loss: 1198.937
Val loss tensor(1753.0731, grad_fn=<DivBackward0>)
[2,    10] loss: 747.797
Val loss tensor(2003.6918, grad_fn=<DivBackward0>)
[2,    20] loss: 663.638
Val loss tensor(1079.5634, grad_fn=<DivBackward0>)
[3,    10] loss: 740.455
Val loss tensor(1072.9314, grad_fn=<DivBackward0>)
[3,    20] loss: 665.277
Val loss tensor(1168.0272, grad_fn=<DivBackward0>)
[4,    10] loss: 504.485
Val loss tensor(700.1854, grad_fn=<DivBackward0>)
[4,    20] loss: 483.855
Val loss tensor(916.5062, grad_fn=<DivBackward0>)
[5,    10] loss: 512.006
Val loss tensor(1098.5355, grad_fn=<DivBackward0>)
[5,    20] loss: 542.469
Val loss tensor(691.7866, grad_fn=<DivBackward0>)


KeyboardInterrupt: 

In [15]:
from scene_network_alexnet import alexnet_siamese as scene_network
i = 90
batch_size = 2
scene_model = scene_network()

uav_input = torch.cat((matching_uav_data[i*(batch_size//2):(i+1)*(batch_size//2)],non_matching_uav_data[i*(batch_size//2):(i+1)*(batch_size//2)]))

sat_input = torch.cat((matching_sat_data[i*(batch_size//2):(i+1)*(batch_size//2)],non_matching_sat_data[i*(batch_size//2):(i+1)*(batch_size//2)]))


labels = torch.tensor(([1]*(batch_size//2))+[0]*(batch_size//2))

distances = scene_model(uav_input,sat_input)

# loss = contrastive_loss(distances, labels)

print(distances)

# print(loss)


tensor([  5.2613, 188.9541], grad_fn=<NormBackward1>)


In [ ]:
from torchvision import datasets, models, transforms
from scene_network_alexnet import alexnet_siamese as scene_network
import torch.optim as optim
from sklearn.utils import shuffle
from scene_network_resnet import scene_network as resnet

# scene_model = resnet("resnet18")
scene_model = scene_network()
optimizer = optim.Adam(scene_model.parameters(), lr=10e-5)

scene_model.train()

uav_validation_data = torch.cat((matching_uav_data_validation,non_matching_uav_data_validation))

sat_validation_data = torch.cat((matching_sat_data_validation,non_matching_sat_data_validation))

validation_labels = torch.tensor([1]*matching_uav_data_validation.size()[0]+[0]*non_matching_uav_data_validation.size()[0])

epochs = 10
batch_size = 16

for epoch in range(epochs):

    matching_uav_data,matching_sat_data = shuffle(matching_uav_data,matching_sat_data)

    non_matching_uav_data,non_matching_sat_data = shuffle(non_matching_uav_data,non_matching_sat_data)

    running_loss = 0.0

    for i in range(len(matching_sat_data)//batch_size):


        optimizer.zero_grad()

        l = torch.tensor(([1]*(batch_size//2))+[0]*(batch_size//2))

        d = scene_model(uav_input,sat_input)


        loss = contrastive_loss(d, l)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
            d_val = scene_model(uav_validation_data,sat_validation_data)

            val_loss = contrastive_loss(d_val,validation_labels)
            print("Val loss", val_loss)

